In [1]:
import pandas as pd

C = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2005_06_social_stata/sec 2a.dta", convert_categoricals=False
)
print(C.shape)
C.sample(5)


(99315, 29)


,hhcode,province,region,psu,idc,s2aq01,s2aq02,s2aq03,s2bq01,s2bq02,...,s2bq12,s2bq13,s2bq14,s2bq15,s2bq16,s2bq17,s2bq18,s2bq19a,s2bq19b,s2bq19c
39487,2.011030e+09,2,1,20110303,2,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
25387,1.161340e+09,1,1,11613401,3,1.0,1.0,1.0,3,NaN,...,2.0,3.0,6.0,1.0,5.0,4.0,2.0,25.0,400.0,425.0
51071,2.061110e+09,2,1,20611101,55,NaN,NaN,NaN,3,NaN,...,3.0,5.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,500.0
32547,1.252030e+09,1,2,12520301,4,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0
58731,2.112010e+09,2,2,21120101,3,1.0,1.0,1.0,2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0


In [2]:
# Create a yes/no column for "ever admitted"
C['ever_admitted']= 'yes'
C.loc[(C["s2bq01"] == 1), 'ever_admitted'] = 'no'
C['currently_enrrolled']= 'no'
C.loc[(C["s2bq01"] == 3), 'currently_enrrolled'] = 'yes'

In [3]:
# Reduce to the questions of interest (4 questions):

C["max_level_acheived"] = C["s2bq05"]
C["why_not"] = C["s2bq02"]
#C['enrolled_last_year']=C["s2bq05"]

C_relevant = C[
    [
        "hhcode",
        "msno"
        "ever_admitted",
        "max_level_acheived",
        "currently_enrrolled",
        "why_not",
        "region",
        "province",
    ]
].copy()


In [4]:
C_relevant

,hhcode,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province
0,1.011010e+09,no,NaN,no,NaN,1,1
1,1.011010e+09,yes,17.0,no,NaN,1,1
2,1.011010e+09,yes,12.0,no,NaN,1,1
3,1.011010e+09,yes,16.0,no,NaN,1,1
4,1.011010e+09,yes,NaN,yes,NaN,1,1
...,...,...,...,...,...,...,...
99310,4.071340e+09,yes,NaN,yes,NaN,1,4
99311,4.071340e+09,yes,8.0,no,NaN,1,4
99312,4.071340e+09,no,NaN,no,NaN,1,4
99313,4.071340e+09,yes,NaN,yes,NaN,1,4


In [5]:
B = pd.read_stata(
    "/Users/clarissaache/Downloads/microdata_2005_06_social_stata//1a_roster.dta"
)
B.sample(5)

,hhcode,idc,s1aq02,s1aq03,s1aq04,age,s1aq05a,s1aq05b,s1aq05c,s1aq06,s1aq07,s1aq08,s1aq09,s1aq10,region,province,psu
5509,1.041020e+09,1,head,male,1,27,12.0,2.0,1978.0,currently married,51.0,not in roster,not in roster,yes,urban,punjab,10410203
6992,1.051020e+09,51,spouse,female,1,40,0.0,0.0,1965.0,currently married,1.0,not in roster,not alive,yes,urban,punjab,10510202
73177,3.021030e+09,53,son/daughter,male,1,7,4.0,9.0,1998.0,unmarried/never married,NaN,not in roster,51,yes,urban,nwfp,30210301
18599,1.111320e+09,2,servants/their relatives,male,1,18,0.0,0.0,1987.0,unmarried/never married,NaN,not in roster,not in roster,yes,urban,punjab,11113201
88400,3.152040e+09,55,grand child,male,1,3,0.0,0.0,2002.0,unmarried/never married,NaN,3,54,yes,rural,nwfp,31520401


In [6]:
# Change the names of the questions for readability
B['sex'] = B['s1aq03']
B['marital_status'] = B['s1aq06']

B_relevant = B[['hhcode', 'msno', 'age', 'sex', 'marital_status']].copy()

In [7]:
# Made the Household Code column a str type
B["hhcode"] = B["hhcode"].astype("str")
C["hhcode"] = C["hhcode"].astype("str")

In [8]:
# MERGE
years05_06 = pd.merge(B_relevant, C_relevant, on=["hhcode"], how='inner', indicator=False)

# Evaluate merge 
# years05_06._merge.value_counts() (ALL GOOD)

In [9]:
# Select people in school enrollment age (I would say 4-10 years old)
years05_06 = years05_06.loc[(years05_06["age"] >= 4)&(years05_06["age"] <= 15)]

In [10]:
years05_06.province.value_counts()
# weird! It doesnt have the provinces with codes 5,6,7

1    104188
3     83419
2     76920
4     54703
Name: province, dtype: int64

In [11]:
# Lets see which hh codes we have:
years05_06["hhcode"] = years05_06["hhcode"].astype("str")
years05_06['subprovince code']= years05_06['hhcode'].apply(lambda x: x[0:4])
years05_06["subprovince code"] = years05_06["subprovince code"].astype("int")
years05_06.loc[(years05_06["region"] == 1), 'region'] = 'urban'
years05_06.loc[(years05_06["region"] == 2), 'region'] = 'rural'

In [12]:
# Now we import the stratum name - code data
strat_name = pd.read_csv(
    "/Users/clarissaache/Documents/IDS 701/stratum0506_v1.csv"
)
strat_name.sample(5)

,province,subprovince code,subprovince,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6
131,NWFP,3061,Bannu,NaN,NaN,NaN,NaN
64,NWFP,3142,Tank,NaN,NaN,NaN,NaN
9,Punjab,1102,Faisalabad,NaN,NaN,NaN,NaN
81,Balochistan,4082,Deara Bughti,NaN,NaN,NaN,NaN
41,Sindh,2072,Shikarpur,NaN,NaN,NaN,NaN


In [13]:
# MERGE
years05_06_ = pd.merge(years05_06, strat_name, on=["subprovince code"], how='inner', indicator=True)

# Evaluate merge 
years05_06_._merge.value_counts()

both          319230
left_only          0
right_only         0
Name: _merge, dtype: int64

In [14]:
years05_06_.sample(10)

,hhcode,age,sex,marital_status,ever_admitted,max_level_acheived,currently_enrrolled,why_not,region,province_x,subprovince code,province_y,subprovince,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,_merge
78787,1232010110.0,12,female,unmarried/never married,yes,8.0,no,NaN,rural,1,1232,Punjab,Vehari,NaN,NaN,NaN,NaN,both
161246,2092020209.0,11,male,unmarried/never married,no,NaN,no,NaN,rural,2,2092,Sindh,Dadu,NaN,NaN,NaN,NaN,both
181161,3011010105.0,11,female,unmarried/never married,yes,NaN,yes,NaN,urban,3,3011,NWFP,Malakand,NaN,NaN,NaN,NaN,both
210540,3062030113.0,4,male,unmarried/never married,yes,10.0,no,NaN,rural,3,3062,NWFP,Malakand,NaN,NaN,NaN,NaN,both
41608,1111140106.0,7,male,unmarried/never married,yes,8.0,no,NaN,urban,1,1111,Punjab,Rawalpindi,NaN,NaN,NaN,NaN,both
51180,1131140111.0,7,female,unmarried/never married,yes,NaN,yes,NaN,urban,1,1131,Punjab,Gujranwala,NaN,NaN,NaN,NaN,both
7285,1022040214.0,6,male,unmarried/never married,yes,NaN,yes,NaN,rural,1,1022,Punjab,Attock,NaN,NaN,NaN,NaN,both
159323,2082040206.0,9,male,unmarried/never married,no,NaN,no,11.0,rural,2,2082,Sindh,Larkana,NaN,NaN,NaN,NaN,both
132185,2051210805.0,4,female,unmarried/never married,yes,5.0,no,NaN,urban,2,2051,Sindh,Karachi,NaN,NaN,NaN,NaN,both
32390,1091220204.0,8,female,unmarried/never married,yes,12.0,no,NaN,urban,1,1091,Punjab,Lahore,NaN,NaN,NaN,NaN,both


In [15]:
years05_06_.to_csv('/Users/clarissaache/Documents/IDS 701/uds-2022-ids-701-team-3/10_data_cleaning/clean_clarissa/years05_06_.', index_label="id")